In [1]:
from pathlib import Path
import optuna
import subprocess
import pickle
import numpy as np

In [2]:
def run_command(command):
    result = subprocess.run(command, capture_output=True)
    assert (
        result.returncode == 0
    ), f"""
command:
{' '.join(command)}
returncode: {result.returncode}
stderr:
{result.stderr.decode()}
stdout:
{result.stdout.decode()}
"""
    return result

In [3]:
def objective(trial):
    global acopp_dir, instance_name, debug_mode

    ants = trial.suggest_int("ants", 10, 500)
    alpha = trial.suggest_float("alpha", 0, 10)
    beta = trial.suggest_float("beta", 0, 10)
    rho = trial.suggest_float("rho", 0, 1)
    q0 = trial.suggest_float("q0", 0, 1)
    ptries = trial.suggest_int("ptries", 1, 5)

    cluster_size = trial.suggest_int("cluster_size", 10, 128)
    sector = trial.suggest_int("sector", 2, cluster_size)
    n_cluster = trial.suggest_int("n_cluster", 2, 1000 // cluster_size + 1)

    command = [
        'python3',
        f'{acopp_dir}/run.py',
        '--acopp_dir',
        str(acopp_dir),
        '--instance_name',
        instance_name,
        '--aaco_nc',
        '--run_only',
        '--experiment',
        '--no_log',
        # '--sol_dir',
        # '../../solutions/temp',
        '--silent',
        '1',
        '--random_seed',
        str(np.random.randint(1,2147483647)),
        '--alpha',
        str(alpha),
        '--beta',
        str(beta),
        '--rho',
        str(rho),
        '--q0',
        str(q0),
        '--ants',
        str(ants),
        '--ptries',
        str(ptries),
        '--sector',
        str(sector),
        '--cluster_size',
        str(cluster_size),
        '--n_cluster',
        str(n_cluster),
    ]
    if debug_mode:
        command += ["--time", "10"]
    
    try:
        result = run_command(command)
        stdout_log = result.stdout.decode()
        profit = int(stdout_log)
    except:
        profit = -1

    return profit

In [4]:
debug_mode = False
acopp_dir = Path("./")
instance_name = 'eil51_10_usw_10_03'
n_trials = 5000
n_jobs = 6
if debug_mode:
    n_trials = 2
    n_jobs = 2
save_path = Path("./study2.pkl")
sampler = optuna.samplers.TPESampler()

In [5]:
command = [
    'python3',
    f'{acopp_dir}/run.py',
    '--acopp_dir',
    str(acopp_dir),
    '--build_only',
    '--experiment'
    ]
result = run_command(command)
print(result.stdout.decode())

$ cmake -DCMAKE_EXPORT_COMPILE_COMMANDS:BOOL=TRUE -G Unix Makefiles -S. -B./build -DCMAKE_BUILD_TYPE:STRING=Release
-- Configuring done
-- Generating done
-- Build files have been written to: /mnt/c/home/vincent/acoplusplus_thop_modified/src/aco++/build



$ make -C ./build
make: Entering directory '/mnt/c/home/vincent/acoplusplus_thop_modified/src/aco++/build'
make[1]: Entering directory '/mnt/c/home/vincent/acoplusplus_thop_modified/src/aco++/build'
make[2]: Entering directory '/mnt/c/home/vincent/acoplusplus_thop_modified/src/aco++/build'
make[2]: Leaving directory '/mnt/c/home/vincent/acoplusplus_thop_modified/src/aco++/build'
make[2]: Entering directory '/mnt/c/home/vincent/acoplusplus_thop_modified/src/aco++/build'
[  9%] Linking CXX executable acothop
make[2]: Leaving directory '/mnt/c/home/vincent/acoplusplus_thop_modified/src/aco++/build'
[100%] Built target acothop
make[1]: Leaving directory '/mnt/c/home/vincent/acoplusplus_thop_modified/src/aco++/build'
make: Leaving directo

In [6]:
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=n_trials, n_jobs=n_jobs)

[I 2023-06-01 19:46:44,870] A new study created in memory with name: no-name-22a1e1d8-23fd-4213-86ac-043668f9ecaa
[I 2023-06-01 19:47:34,291] Trial 5 finished with value: 198807.0 and parameters: {'ants': 329, 'alpha': 9.673961978387474, 'beta': 9.204772429061393, 'rho': 0.6771918545995784, 'q0': 0.9955374186139019, 'ptries': 2, 'cluster_size': 31, 'sector': 17, 'n_cluster': 26}. Best is trial 5 with value: 198807.0.
[I 2023-06-01 19:47:34,300] Trial 1 finished with value: 204820.0 and parameters: {'ants': 123, 'alpha': 3.7378819011096764, 'beta': 9.208123184677072, 'rho': 0.43411102081039654, 'q0': 0.6968474583062607, 'ptries': 2, 'cluster_size': 124, 'sector': 24, 'n_cluster': 7}. Best is trial 1 with value: 204820.0.
[I 2023-06-01 19:47:34,305] Trial 0 finished with value: 210237.0 and parameters: {'ants': 329, 'alpha': 6.473185582718579, 'beta': 4.79616293180687, 'rho': 0.0034022593213907104, 'q0': 0.33337330884575944, 'ptries': 2, 'cluster_size': 120, 'sector': 112, 'n_cluster': 4

In [7]:
with open(save_path, "wb") as f:
    pickle.dump(study, f)